In [ ]:
import multiprocessing
from multiprocessing import Process, Queue

import time
from lxml import etree
import requests
from bs4 import BeautifulSoup

url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1'
infos = []
def check_req_url(url): #測試請求網址是否請求成功
    r = requests.get(url) #請求網址
    #print(resp.status_code) #錯誤時404,成功時200
    if r.status_code != 200:  #如果請求失敗
        print('Invalid url:', r.url) #印出請求失敗的網址
        return "fail" #回傳失敗提示訊息
    else:
        return r.text #回傳請求成功的html文字

def get_week_new_movies(webpage): #抓取電影資訊
  soup = BeautifulSoup(webpage) #網頁解析
  for d in soup.find_all('div', class_="release_info"):
      info = {}
      info['中文名稱'] = d.find('div', class_='release_movie_name').find('a', class_='gabtn').text.strip()
      info['英文名稱'] = d.find('div', class_='release_movie_name').find('div', class_='en').a.text.strip()
      info['上映日期'] = d.find('div', class_='release_movie_time').text.split('上映日期 ： ')[1]
      info['期待度'] = ''
      info['劇情介紹'] = d.find('div', class_='release_text').find('span', class_='jq_text_overflow_180 jq_text_overflow_href_list').text.strip()
      #info['分類'] = 
      if d.find('div', class_='leveltext'):    
          info['期待度'] = d.find('div', class_='leveltext').span.text
      infos.append(info) 
  return infos

if __name__ == '__main__':
  webpage = check_req_url(url)
  if webpage:
    infos = get_week_new_movies(webpage)
  for info in infos:
    print(info)


{'中文名稱': '小小兵2：格魯的崛起', '英文名稱': 'Minions: The Rise of Guru', '上映日期': '2022-07-01', '期待度': '93%', '劇情介紹': '今年暑假，影史最賣座的動畫電影系列之一和風靡全球的現象，即將帶來關於一個12歲小男孩的故事，他夢想成為世上最偉大的超級大壞蛋。這部全新動畫喜劇動作片就是《小小兵2：格魯的崛起》。\r\n\xa0\r\n這部電影充滿了照明娛樂招牌的顛覆性和無厘頭喜劇橋段、各種大眾文化的笑梗、感人肺腑的劇情、勁爆的電影配樂，以及宇宙超級無敵誇張的動作場面。《小小兵2：格魯的崛起》一片由這個電影系列的原創掌舵，製片包括照明娛樂極具遠見的創辦人和執行長克里斯梅勒丹德利、他的長期合作夥伴珍娜海莉，以及克里斯雷納德，導演凱爾巴爾達（《神偷奶爸3》、《小小兵》）則再度回歸執導這部電影。共同導演是布萊德艾柏森（長壽喜劇影集《辛普森家庭》）以及強納森戴凡（《寵物當家》系列電影）。'}
{'中文名稱': '蝙蝠俠', '英文名稱': 'The Batman', '上映日期': '2022-03-04', '期待度': '92%', '劇情介紹': '《蝙蝠俠》將聚焦於一個年輕版本的布魯斯韋恩，並從中檢視他在面罩之下的靈魂。羅伯派汀森將在新版《蝙蝠俠》中飾演這位DC超級英雄，電影是由《猩球崛起：終極決戰》導演麥特李維斯（Matt Reeves）所執導。'}
{'中文名稱': '魔比斯', '英文名稱': 'Morbius', '上映日期': '2022-01-16', '期待度': '86%', '劇情介紹': '★索尼與漫威繼《猛毒》後再一野心暗黑爭議之作\xa0\r\n★《蜘蛛人》宇宙正式成立 全新神秘反派 吸血鬼「魔比斯」首推獨立電影\r\n★奧斯卡得主傑瑞德雷托領軍 金球獎影麥可基頓火力加持 再創漫威宇宙高峰\r\n\xa0\r\n在漫威宇宙中最具爭議性且複雜的角色即將在大銀幕與全球觀眾見面！由奧斯卡得主傑瑞德雷托領軍，飾演一位患有罕見疾病的生物化學博士麥可魔比斯；為治療自己的疾病，並試圖挽救其他遭受相同命運的人，他試圖進行一場絕望的豪賭，並大膽在實驗過程中利用吸血蝙蝠的DNA治療。雖說實驗結果在起初看似成功，但最後的結果只是顯示，這一切可能只是帶給他比原有疾病更糟糕的痛苦而已……'}
{

In [ ]:
import multiprocessing
from multiprocessing import Process, Queue

import time
from lxml import etree
import requests
from bs4 import BeautifulSoup

url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1'
infos = []
def check_req_url(url): #測試請求網址是否請求成功
    r = requests.get(url) #請求網址
    #print(resp.status_code) #錯誤時404,成功時200
    if r.status_code != 200:  #如果請求失敗
        print('Invalid url:', r.url) #印出請求失敗的網址
        return "fail" #回傳失敗提示訊息
    else:
        return r.text #回傳請求成功的html文字

def get_week_new_movies(webpage): #抓取電影資訊
  soup = BeautifulSoup(webpage) #網頁解析
  for d in soup.find_all('div', class_="release_info"):
      info = {}
      info['中文名稱'] = d.find('div', class_='release_movie_name').find('a', class_='gabtn').text.strip()
      info['英文名稱'] = d.find('div', class_='release_movie_name').find('div', class_='en').a.text.strip()
      info['上映日期'] = d.find('div', class_='release_movie_time').text.split('上映日期 ： ')[1]
      info['期待度'] = ''
      info['劇情介紹'] = d.find('div', class_='release_text').find('span', class_='jq_text_overflow_180 jq_text_overflow_href_list').text.strip()
      #info['分類'] = 
      
      if d.find('div', class_='leveltext'):    
          info['期待度'] = d.find('div', class_='leveltext').span.text
      infos.append(info) 
  return infos

if __name__ == '__main__':
  webpage = check_req_url(url)
  if webpage:
    infos = get_week_new_movies(webpage)
  for info in infos:
    print(info)


[{'上映日期': '2021-08-11',
  '中文名稱': '玩命關頭9',
  '劇情介紹': '馮迪索飾演的唐老大和莉蒂與他的兒子小布萊恩，正在過著與世隔絕的平靜生活，但是他們都知道這種平靜的生活隨時隨地都會受到致命危機的威脅。這次，這個威脅將逼迫唐老大面對他過去犯下的罪孽，如果他想要拯救他最愛的人們。他的團隊將再度攜手合作，槓上他們遇過最高竿的職業刺客和飆速車手，並且阻止他摧毀全世界的邪惡計畫。這個神秘人物也正好是早就被唐老大切斷兄弟關係的親弟弟雅各（約翰希南 飾）。\r\n\xa0\r\n【關於電影】\r\n曾經執導《玩命關頭》第3集、第4集、第5集和第6集的華裔導演林詣彬回歸執導《玩命關頭9》，他讓這個電影系列成為全球大賣的超級動作鉅片。《玩命關頭9》這部全新系列電影中的動作場面橫跨全球，從倫敦到東京，從中美洲到愛丁堡，然後從位於亞塞拜然的秘密地下碉堡，一直到喬治亞共和國首都提比里斯車水馬龍、熙來攘往的街頭。這一路上，老朋友將起死回生，昔日的敵人將再度回歸，歷史將被重新改寫，而且家庭真正的意義也將受到前所未有的考驗。再度回歸參與這部電影演出的堅強卡司包括蜜雪兒羅卓奎茲、泰瑞斯吉布森、路達克里斯、喬丹娜布魯斯特、娜塔莉艾曼紐、姜成鎬，以及奧斯卡影后海倫米蘭與莎莉賽隆。曾經榮獲葛萊美獎肯定的當紅歌手和超級巨星卡蒂B在《玩命關頭9》一片中將飾演全新角色蕾莎，一個和唐老大的過去有關聯的女子，另外，雷鬼動音樂巨星歐蘇納也在這部電影中客串演出。',
  '期待度': '92%',
  '英文名稱': 'Fast & Furious 9'},
 {'上映日期': '2021-08-12',
  '中文名稱': '脫稿玩家',
  '劇情介紹': '《脫稿玩家》 故事描述一個銀行小職員 - 蓋伊 (萊恩雷諾斯) 日復一日年復一年做著無聊的工作，過著樸實無華的生活，直到有一天他突然發現自己是一個暴力電玩中可有可無的配角，他想要突破現狀，但這個動作竟讓整個遊戲大變調，除了要挑戰層層關卡，還要拯救世界，更重要的是否能從C咖一躍成A咖呢？',
  '期待度': '100%',
  '英文名稱': 'Original title: Free Guy'},
 {'上映日期': '2021-08-13',
  '中文名稱': '被偷走的童年',
  '劇情介紹': '★《何處是

In [ ]:
#目標位置>>Yahoo>>電影>>Yahoo本週新片
#https://tw.movies.yahoo.com/movie_thisweek.html

import requests
import re
from bs4 import BeautifulSoup

#Yahoo電影
yahoo_movie_url = 'https://tw.movies.yahoo.com/movie_thisweek.html' #目標位置

def check_req_url(url): #測試請求網址是否請求成功
    resp = requests.get(url) #請求網址
    #print(resp.status_code) #錯誤時404,成功時200
    if resp.status_code != 200:  #如果請求失敗
        print('Invalid url:', resp.url) #印出請求失敗的網址
        return "fail" #回傳失敗提示訊息
    else:
        return resp.text #回傳請求成功的html文字

def get_week_new_movies(webpage): #抓取電影資訊
    soup = BeautifulSoup(webpage, 'html5lib') #網頁解析
    movies = [] #預設電影資訊存這裡 
   
    #抓取<div class="release_info_text"></div>內文字
    rows = soup.find_all('div', 'release_info_text')
    
    #print(rows)
    for row in rows:
        data_movie = dict() #存成{"key":"value"}格式
        #電影名稱
        data_movie['ch_name'] = row.find('div', 'release_movie_name').a.text.strip()
        #英文名稱
        data_movie['english_name'] = row.find('div', 'release_movie_name').find('div', 'en').a.text.strip()
        #電影介紹
        data_movie['info'] = row.find('div', 'release_text').text.strip()
        #期待度
        data_movie['expectation'] = row.find('div', 'leveltext').span.text.strip()
        #上映日期 只抓日期 "上映日期：2020-11-20" -> match.group(0): "2020-11-20"
        data_movie['release_date'] = get_date(row.find('div', 'release_movie_time').text)

        movies.append(data_movie) #再被取代前先存入for 外面的movies=[]
       
    return movies

def get_date(date_str):
    # e.x. "上映日期：2017-03-23" -> match.group(0): "2020-11-20"
    #記得import re
    #1.\d找到數字
    #2.要找一整組的所以\d+，+代表1個以上
    #3.有可能有'-'

    pattern = '\d+-\d+-\d+'
    match = re.search(pattern, date_str)
    #print(match)
    #print(match.group(0))
    
    if match is None:
        return date_str
    else:
        return match.group(0)

if __name__ == '__main__':
    webpage = check_req_url(yahoo_movie_url)
    #print(webpage)
    
    if webpage:
        movies = get_week_new_movies(webpage)
        print(movies)

[{'ch_name': '玩命關頭9', 'english_name': 'Fast & Furious 9', 'info': '馮迪索飾演的唐老大和莉蒂與他的兒子小布萊恩，正在過著與世隔絕的平靜生活，但是他們都知道這種平靜的生活隨時隨地都會受到致命危機的威脅。這次，這個威脅將逼迫唐老大面對他過去犯下的罪孽，如果他想要拯救他最愛的人們。他的團隊將再度攜手合作，槓上他們遇過最高竿的職業刺客和飆速車手，並且阻止他摧毀全世界的邪惡計畫。這個神秘人物也正好是早就被唐老大切斷兄弟關係的親弟弟雅各（約翰希南 飾）。\n\xa0\n【關於電影】\n曾經執導《玩命關頭》第3集、第4集、第5集和第6集的華裔導演林詣彬回歸執導《玩命關頭9》，他讓這個電影系列成為全球大賣的超級動作鉅片。《玩命關頭9》這部全新系列電影中的動作場面橫跨全球，從倫敦到東京，從中美洲到愛丁堡，然後從位於亞塞拜然的秘密地下碉堡，一直到喬治亞共和國首都提比里斯車水馬龍、熙來攘往的街頭。這一路上，老朋友將起死回生，昔日的敵人將再度回歸，歷史將被重新改寫，而且家庭真正的意義也將受到前所未有的考驗。再度回歸參與這部電影演出的堅強卡司包括蜜雪兒羅卓奎茲、泰瑞斯吉布森、路達克里斯、喬丹娜布魯斯特、娜塔莉艾曼紐、姜成鎬，以及奧斯卡影后海倫米蘭與莎莉賽隆。曾經榮獲葛萊美獎肯定的當紅歌手和超級巨星卡蒂B在《玩命關頭9》一片中將飾演全新角色蕾莎，一個和唐老大的過去有關聯的女子，另外，雷鬼動音樂巨星歐蘇納也在這部電影中客串演出。', 'expectation': '92%', 'release_date': '2021-08-11'}, {'ch_name': '脫稿玩家', 'english_name': 'Original title: Free Guy', 'info': '《脫稿玩家》 故事描述一個銀行小職員 - 蓋伊 (萊恩雷諾斯) 日復一日年復一年做著無聊的工作，過著樸實無華的生活，直到有一天他突然發現自己是一個暴力電玩中可有可無的配角，他想要突破現狀，但這個動作竟讓整個遊戲大變調，除了要挑戰層層關卡，還要拯救世界，更重要的是否能從C咖一躍成A咖呢？', 'expectation': '100%', 'release_date': '2021-08-12'}, {'ch_name': '被偷走的童年', 'englis

In [ ]:
#!/usr/bin/env python2
# -*- coding=utf-8 -*-
import multiprocessing
from multiprocessing import Process, Queue

import time
from lxml import etree
import requests


class DouBanSpider(Process):
    def __init__(self, q, url_list, lock):
        # 重寫寫父類的__init__方法
        super(DouBanSpider, self).__init__()
        self.url_list = url_list
        self.q = q
        self.lock = lock
        self.headers = {
            'Host': 'movie.douban.com',
            'Referer': 'https://movie.douban.com/top250?start=225&filter=',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.104 Safari/537.36',
        }

    def run(self):
        self.parse_page()

    def send_request(self, url):
        '''
        用來傳送請求的方法
        :return: 返回網頁原始碼
        '''
        # 請求出錯時，重複請求３次,
        i = 0
        while i <= 3:
            try:
                print(u"[INFO]請求url:" + url)
                return requests.get(url=url, headers=self.headers).content
            except Exception as e:
                print(u'[INFO] %s%s' % (e, url))
                i += 1

    def parse_page(self):
        '''
        解析網站原始碼，並採用ｘｐａｔｈ提取　電影名稱和平分放到佇列中
        :return:
        '''
        time.sleep(0.1)
        while 1:
            try:

                url = self.url_list.pop()
            except IndexError as e:
                break
            self.lock.acquire()
            response = self.send_request(url)
            html = etree.HTML(response)
            # 　獲取到一頁的電影資料
            node_list = html.xpath("//div[@class='info']")
            for move in node_list:
                # 電影名稱
                title = move.xpath('.//a/span/text()')[0]
                # 評分
                score = move.xpath('.//div[@class="bd"]//span[@class="rating_num"]/text()')[0]

                # 將每一部電影的名稱跟評分加入到佇列
                self.q.put(score + "\t" + title)
            self.lock.release()


class AllUrlSpider(Process):
    def __init__(self, url_lis):
        super(AllUrlSpider, self).__init__()
        self.url_list = url_lis

    def run(self):
        base_url = 'https://movie.douban.com/top250?start='
        # 構造所有ｕｒｌ
        for num in range(225, -1, -25):
            self.url_list.append(base_url + str(num))
            print("獲得URL：{}".format(base_url + str(num)))


def main():
    # 建立一個佇列用來儲存程式獲取到的資料
    q = Queue()
    lock = multiprocessing.Lock()

    manager = multiprocessing.Manager()
    url_list = manager.list()
    a = AllUrlSpider(url_list)

    p = DouBanSpider(q, url_list, lock)
    b = DouBanSpider(q, url_list, lock)
    c = DouBanSpider(q, url_list, lock)

    a.start()
    p.start()
    b.start()
    c.start()

    a.join()
    p.join()
    b.join()
    c.join()

    while not q.empty():
        print(q.get())


if __name__ == "__main__":
    start = time.time()
    main()
    print('[info]耗時：%s' % (time.time() - start))

獲得URL：https://movie.douban.com/top250?start=225
獲得URL：https://movie.douban.com/top250?start=200
獲得URL：https://movie.douban.com/top250?start=175
獲得URL：https://movie.douban.com/top250?start=150
獲得URL：https://movie.douban.com/top250?start=125
獲得URL：https://movie.douban.com/top250?start=100
獲得URL：https://movie.douban.com/top250?start=75
獲得URL：https://movie.douban.com/top250?start=50
獲得URL：https://movie.douban.com/top250?start=25
獲得URL：https://movie.douban.com/top250?start=0
[INFO]請求url:https://movie.douban.com/top250?start=0
[INFO]請求url:https://movie.douban.com/top250?start=25
[INFO]請求url:https://movie.douban.com/top250?start=50
[INFO]請求url:https://movie.douban.com/top250?start=75
[INFO]請求url:https://movie.douban.com/top250?start=100
[INFO]請求url:https://movie.douban.com/top250?start=125
[INFO]請求url:https://movie.douban.com/top250?start=150
[INFO]請求url:https://movie.douban.com/top250?start=175
[INFO]請求url:https://movie.douban.com/top250?start=200
[INFO]請求url:https://movie.douban.com/top250?

In [ ]:
from bs4 import BeautifulSoup
import requests
 
for page in range(1, 4):  # 執行1~3頁
 
    # 連結網站
    response = requests.get(
        "https://www.inside.com.tw/tag/AI?page=" + str(page))
 
    # HTML原始碼解析
    soup = BeautifulSoup(response.text, "html.parser")
 
    # 取得所有class為post_title的<h3>
    titles = soup.find_all("h3", {"class": "post_title"})
 
    print(f"====================第{str(page)}頁====================")
    for title in titles:
        print(title.select_one("a").getText())  # 取得連結的標題文字

====================第1頁====================
iKala 十週年記者會：疫情衝擊擴張版圖 年營收預計突破十億元台幣
【Howie 商業投資】追上 Google！ 臉書推出 Droidlet 加速智慧機器人開發
【開放文化觀點】難用、歧視又卸責！美國政府的人臉辨識服務如何辜負成千上萬失業者？
【塞掐 Side Chat】EP 23｜進了人人稱羨的超大企業，為何離職創業？ft. AI 新創 A Team
新創助傳統產業變智慧！SparkLabs Taipei DemoDay 5 聚焦數位轉型
橫跨軟硬體與解決方案，研華董事長劉克振親揭 2025 AIoT 佈局
Appier Q2 財報：營收 28 億日圓 YoY 增 50%，並於 6 月實現首次營業獲利
科技部 AI 教練助攻！東奧舉重郭婞淳奪金背後的視覺分析工具
IN 啦！「鷹眼系統」到底是什麼科技？
【電車世代】安永 AI 推測：全球主要市場電動車銷量將在 2033 超越油車
高通攜手富士康推出基於 Qualcomm Cloud AI 100 的 Gloria AI Edge Box 邊際 AI 設備，獲越南 BKAV 採納並用於智慧邊際
Computex 2021 ： Intel 強調 AI 以 XPU 架構為核心戰略，以 Xeon 為核心搭配異構運算解決複雜而廣泛的運算問題
高通在 Snapdragon 888 展示基於軟體的 AI 影片解碼器，可達成 1,280 x 704 30fps 的即時解碼
NVIDIA 遊戲夥伴夏日迎接 RTX 光追與 DLSS 加持，以樂高 拼砌旅程、毀滅戰士 永恆為首的新遊戲將獲得絕佳光影效果
全球首創認知智慧超高畫質顯示器 Sony BRAVIA XR 系列仿人腦高效分析運作呈現極致影像
NVIDIA 針對工業應用推出工規級 Jetson AGX Xavier 工業模組，具備寬溫耐性與抗衝擊與振動特性
韓國AI女團Eternity讓你怕怕的嗎？如何避開「恐怖谷現象」，降低不舒服感受？
Google 採用 AI 技術設計新一代 TPU ，僅需 6 小時就達到人類需要花費數個月的成果
Computex 2021 ： NVIDIA 邀請台灣智慧駕駛、勤崴國際分享台灣當前自動駕駛的產業現況
====================第2頁====